In [1]:
#FIRST COLLECT TRAINED MODELS AND ITS RESULTS FROM 10 DIFFERENT HOSPITALS. TRAIN YOUR NEURAL NETWORK ON THOSE MODELS WITH 10
#DIFFERENT LABELS FOR EACH IMAGE. TO PREDICT THE OUTPUT USE A MAX FUNCTION, DIFFERENTIALLY PRIVATE QUERY TO SEE TRUE OUTPUT.
#WE ALSO NEED TO ADD NOISE TO THE DATABASE SO AS TO SAFEGUARD DETAILS OF PEOPLE IN THE DATASET. 
#NEXT, RETRAIN A NEW MODEL ON YOUR OWN LOCAL DATASET THAT HAS THE LABELS.

#THIS FILE CONSISTS OF ONLY THE DIFFERENTIALLY PRIVATE QUERIES PART

In [2]:
import numpy as np

In [5]:
num_teachers = 10 #cuz we partnered up with 10 hospitals
num_examples = 10000 #this is our dataset
num_labels = 10 #labels for our classifier


In [31]:
preds = (np.random.rand(num_teachers, num_examples) * num_labels).astype(int).transpose(1,0) #fake predictions fr example here
#we need 10 sets of 10000 predictions categorised into 10 different labels. so the abv query.
#transpose(1,0) shudnt be used for demos below

In [9]:
#DEMO
preds
preds[0] #predictions from one teacher for all 10000 imgs on our dataset
preds[0].shape #check if it is 10000 cuz we have 1 prediction for each img in our dataset

(10000,)

In [12]:
preds[:,0] # we have 10 vals fr each img tht came frm 10 different teachers.1st val frm 1 teacher, 2nd val frm 2nd teacher etc

array([2, 5, 5, 7, 0, 3, 2, 5, 9, 4])

In [15]:
an_image = preds[:, 0]
an_image

array([2, 5, 5, 7, 0, 3, 2, 5, 9, 4])

In [16]:
label_counts = np.bincount(an_image, minlength=num_labels)#returns count of each variable
label_counts

array([1, 0, 2, 1, 1, 3, 0, 1, 0, 1], dtype=int64)

In [17]:
np.argmax(label_counts) #gives the label which is repeated max no of times

5

In [18]:
epsilon = 0.1 #argmax(label_counts) isnt necessarily differentially private. so we add laplacian noise as well
beta = 1/epsilon

for i in range(len(label_counts)):
    label_counts[i] += np.random.laplace(0, beta, 1)

In [19]:
label_counts

array([-19,  19,   3,  -5,   9,  -5,   0, -21, -22, -27], dtype=int64)

In [21]:
new_label = np.argmax(label_counts) #noise makes us get the wrong answer...but somehow we must be ok with tht!!!
#we assume tht in later stages of training in the deep nn, part of noise is filtered out and we reach a value tht is agreed upon
#by most of the hospital models

In [22]:
#COMBINING ALL ABOVE CODE TO ITERATE OVER EACH IMAGE WHICH HAS 10 VALUES EACH

In [32]:
new_labels = list()
for an_image in preds:
    label_counts = np.bincount(an_image, minlength = num_labels)
    
    epsilon = 0.1
    beta = 1/epsilon
    
    for i in range(len(label_counts)):
        label_counts[i] += np.random.laplace(0,beta,1)
        
    new_label = np.argmax(label_counts)
    
    new_labels.append(new_label)

In [33]:
#DEMO
len(new_labels)
new_labels

10000

In [36]:
#****************************************************************************************************************

In [37]:
#PATE ANALYSIS

In [38]:
#****************************************************************************************************************

In [40]:
labels = np.array([9,9,3,6,9,9,9,9,8,2]) #random labels for example
counts = np.bincount(labels, minlength = 10) #counts will gv frequency of each label 0-9
query_result = np.argmax(counts) #gives max repeated label
query_result

#definition of DP is even if we remove one value here one hospital, the query shudnt be affected. in this case, it depends on d
#value of labels. If all hospitals agreed tht it was 9, den query wud hv low sensitivity. but such isnt the case. 

#Also, the value 9 or 1 or any other in the labels doesnt stand for one person but as a collective set of all persons in 1 hosp.
#so removing one person wudnt matter much nd nt cause much change to query output when all hosps agree on 9 or even if we remove
#1 value 9 the output remains 9 itself. if we r able to do this for all labels in the array, den it is a perfect query !!!

#core assumption is tht no person went to 2 hospitals. even if a person went to all the 10 hosps, then removing him wud cause a 
#slight change in all hospitals' labels which wudnt matter much in the end cuz sensitivity is almost same. Hence this query is
#immune to one person's data being removed

9

In [42]:
from syft.frameworks.torch.differential_privacy import pate

ImportError: Traceback (most recent call last):
  File "c:\users\pc\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "c:\users\pc\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "c:\users\pc\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "c:\users\pc\appdata\local\programs\python\python37\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "c:\users\pc\appdata\local\programs\python\python37\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed with error 

In [44]:
num_teachers, num_examples, num_labels = (100,100,10)
preds=(np.random.rand(num_teachers, num_examples) * num_labels).astype(int) #fake predictions
indices = (np.random.rand(num_examples)*num_labels).astype(int) #true answers

In [45]:
data_dependent_epsilon,data_inde_eps = pate.perform_analysis(teacher_preds=preds,true_indices=indices,noise_eps=0.1,delta=1e-5)
print("Data independent epsilon:", data_inde_eps)
print("Data dependent epsilon:", data_dependent_epsilon)
#TENSORFLOW not working...u will get output as (11.756462732485105, 11.765462732485115) for gvn synthetic input
#values r very close but we can see data_indep is teeny weeny bit higher than data_dependent...indicates there is teeny amt of
#agreement between the models.

#this is bcz we generated random inputs nd hence the models dnt agree

NameError: name 'pate' is not defined

In [ ]:
#if we take some strting 5 values to be same, then dep shows approx 8 and indep shows 11 which means dataset leaked only 8 eps
#as opposed to 11 we had to leak before

preds[:,0:5] *= 0
data_dependent_epsilon,data_inde_eps = pate.perform_analysis(teacher_preds=preds,true_indices=indices,noise_eps=0.1,delta=1e-5)
print("Data independent epsilon:", data_inde_eps)
print("Data dependent epsilon:", data_dependent_epsilon)

In [ ]:
#if we make frst 50 values as 0. we have only 100 examples so this is aggressive. might show warning regarding moments variable.
#add moments to pate.perform()
#we get o/p as approx 11.7 indep and 1.52 dep. significantly better privacy leak der
preds[:,0:50] *= 0
data_dep_eps,data_inde_eps=pate.perform_analysis(teacher_preds=preds,true_indices=indices,noise_eps=0.1,delta=1e-5,moments=20)
print("Data independent epsilon:", data_inde_eps)
print("Data dependent epsilon:", data_dep_eps)

#greater the agreement, tighter the epsilon value for dependent. 